In [1]:
import boto3
from botocore.client import Config

from tqdm import tqdm
import yaml

import RS_images_to_S3


Define constants

In [3]:
with open('config.yaml') as fp:
    cfg=yaml.load(fp, Loader=yaml.FullLoader)['RS_images_to_S3.py']

ACCESS_KEY=cfg['access_key']
SECRET_KEY=cfg['secret_key']
ENDPOINT_URL=cfg['url']
BUCKET=cfg['bucket']


Define session

In [4]:
FOLDER_TO_FIX='02_Data/initial/images_RS/'

session = boto3.session.Session()
s3_client = session.client(
    service_name='s3',
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    endpoint_url=ENDPOINT_URL,
    config=Config(s3={'addressing_style': 'path'})
)


In [5]:
response = s3_client.list_objects_v2(Bucket=BUCKET, Prefix=FOLDER_TO_FIX)
files_in_folder = response.get("Contents")


Copy the files of the tiles 1168 and 1228 in a sub folder

In [6]:
files_to_fix=[file['Key'].split('/')[-1] for file in files_in_folder if file['Key'].split('/')[-1].startswith('1168_1228')]

if False:
    files_to_fix=files_to_fix[-3:]


In [7]:
print(files_to_fix[0:5])

[]


In [ ]:
SUBFOLDER_NEW_TILES=FOLDER_TO_FIX+'1168_1228/'

old_filenames=[FOLDER_TO_FIX+filename for filename in files_to_fix]
# print(old_filenames)


In [ ]:
file = old_filenames[0]
new_filename=file.split('/')[-1][9:]

print(new_filename)


In [ ]:
success=[]
for file in tqdm(old_filenames, desc='Moving files for the 2 last tiles'):
    new_filename = SUBFOLDER_NEW_TILES + file.split('/')[-1][9:]
    success.append(RS_images_to_S3.move_files(s3_client, file, BUCKET, new_filename=new_filename))


In [ ]:
print(f"{sum(success)} files were successfully moved or already existed in the folder '{SUBFOLDER_NEW_TILES}'")

Copy the files left in a subfolder

In [8]:
response = s3_client.list_objects_v2(Bucket=BUCKET, Prefix=FOLDER_TO_FIX)
files_in_folder = response.get("Contents")
files_to_fix=[file['Key'].split('/')[-1] for file in files_in_folder 
            if (len(file['Key'].split('/'))<5) and (file['Size']>0)]

if False:
    files_to_fix=files_to_fix[0:5]


In [9]:
print(files_to_fix[0:5])


[]


In [10]:
SUBFOLDER_OLD_TILES=FOLDER_TO_FIX+'1188_1208/'

In [11]:
success=[]
for file in tqdm(files_to_fix, desc='Moving files for the 2 previous tiles'):
    old_filename = FOLDER_TO_FIX + file
    new_filename = SUBFOLDER_OLD_TILES + file
    success.append(RS_images_to_S3.move_files(s3_client, old_filename, BUCKET, new_filename=new_filename))

Moving files for the 2 previous tiles: 0it [00:00, ?it/s]


In [12]:
print(f"{sum(success)} files were successfully moved or already existed in the folder '{SUBFOLDER_OLD_TILES}'")

0 files were successfully moved or already existed in the folder '02_Data/initial/images_RS/1188_1208/'


In [13]:
old_filename

NameError: name 'old_filename' is not defined